In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.tsv', sep='\t', header=None, names=["hate", "offense", "text"])

In [3]:
df.head()

,hate,offense,text
0,0,0,"Alarm set, early bird n all that. Night all xx"
1,1,2,@EvanBecker513 @_TriggaPlease_ @MorbidMermaid ...
2,0,3,RT @_MindAtEase: Wet pussy with no smell a mf
3,0,3,RT @itsSamCollins: I hate seeing people with m...
4,0,3,&#8220;@RawrItsTaraa: Trust these bitches ? Na...


In [4]:
# Word tokenization
from spacy.lang.en import English
from spacy import displacy
import en_core_web_sm

# Load English tokenizer, tagger, parser, NER and word vectors
# nlp = English()
nlp = en_core_web_sm.load()

text = u"""New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases.

At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday.

The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000."""

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

entities=[(i, i.label_, i.label) for i in my_doc.ents]
entities

['New', 'York', 'City', 'on', 'Tuesday', 'declared', 'a', 'public', 'health', 'emergency', 'and', 'ordered', 'mandatory', 'measles', 'vaccinations', 'amid', 'an', 'outbreak', ',', 'becoming', 'the', 'latest', 'national', 'flash', 'point', 'over', 'refusals', 'to', 'inoculate', 'against', 'dangerous', 'diseases', '.', '\n\n', 'At', 'least', '285', 'people', 'have', 'contracted', 'measles', 'in', 'the', 'city', 'since', 'September', ',', 'mostly', 'in', 'Brooklyn', '’s', 'Williamsburg', 'neighborhood', '.', 'The', 'order', 'covers', 'four', 'Zip', 'codes', 'there', ',', 'Mayor', 'Bill', 'de', 'Blasio', '(', 'D', ')', 'said', 'Tuesday', '.', '\n\n', 'The', 'mandate', 'orders', 'all', 'unvaccinated', 'people', 'in', 'the', 'area', ',', 'including', 'a', 'concentration', 'of', 'Orthodox', 'Jews', ',', 'to', 'receive', 'inoculations', ',', 'including', 'for', 'children', 'as', 'young', 'as', '6', 'months', 'old', '.', 'Anyone', 'who', 'resists', 'could', 'be', 'fined', 'up', 'to', '$', '1,00

[(New York City, 'GPE', 382),
 (Tuesday, 'DATE', 388),
 (At least 285, 'CARDINAL', 394),
 (September, 'DATE', 388),
 (Brooklyn, 'GPE', 382),
 (Williamsburg, 'GPE', 382),
 (four, 'CARDINAL', 394),
 (Zip, 'PERSON', 378),
 (Bill de Blasio, 'PERSON', 378),
 (Tuesday, 'DATE', 388),
 (Orthodox, 'NORP', 379),
 (Jews, 'NORP', 379),
 (6 months old, 'DATE', 388),
 (1,000, 'MONEY', 391)]

In [5]:
displacy.render(my_doc, style = "ent",jupyter = True)

In [6]:
type(my_doc)

spacy.tokens.doc.Doc

In [7]:
import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

#Printing the total number of stop words:
print('Number of stop words: %d' % len(spacy_stopwords))

#Printing first ten stop words:
print('First ten stop words: %s' % list(spacy_stopwords)[:20])

Number of stop words: 305
First ten stop words: ['before', 'whoever', 'moreover', 'of', 'just', 'such', 'but', 'give', 'they', 'while', 'meanwhile', 'ours', 'only', 'take', 'you', 'to', 'whether', 'toward', 'yours', 'front']


# Text Classificataion

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    sentence = sentence.strip().lower()
    
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Load Data
df = pd.read_csv('../data/train.tsv', sep='\t', header=None, names=["hate", "offense", "text"])
df.head()

,hate,offense,text
0,0,0,"Alarm set, early bird n all that. Night all xx"
1,1,2,@EvanBecker513 @_TriggaPlease_ @MorbidMermaid ...
2,0,3,RT @_MindAtEase: Wet pussy with no smell a mf
3,0,3,RT @itsSamCollins: I hate seeing people with m...
4,0,3,&#8220;@RawrItsTaraa: Trust these bitches ? Na...


In [9]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x121579c20>,
                vocabulary=None)

In [10]:
X = df['text'] # the features we want to analyze
y = df['hate'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Vectorizer which uses the Spacy Tokenizer
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

# Logistic Regression Classifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC

classifier = DecisionTreeRegressor() #LogisticRegression(max_iter=512)# LogisticRegression(max_iter=512)

# Create pipeline using Bag of Words
pipe = Pipeline([#('cleaner', predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x121579c20>,
                                 vocabulary=None)),
                ('classifier',
                 DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                       max_depth=None, max_features=None,
                                       max_leaf_nodes=None,
                       

In [11]:
predicted = np.rint(pipe.predict(X_test))
print(predicted[0:100])

from sklearn import metrics
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='micro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='micro'))

NameError: name 'np' is not defined

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(predicted[0:10])
bins = np.concatenate(([min(predicted) - 1], bins, [max(predicted) + 1]), axis=None)

predicted = np.digitize(predicted, bins)
print(predicted[0:10])
print(y_test[0:10])

from sklearn import metrics
# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='micro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='micro'))

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

print(predicted[0:20])

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='micro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='micro'))

In [ ]:
(y_test == predicted)[100:1000]

In [ ]:
X_test

In [ ]:
from scipy.stats import pearsonr

In [ ]:
pearsonr(y_test, predicted)

In [ ]:
pearsonr([1,2,3,4,5], [3,4,6,8,10])

In [ ]:
a = pearsonr(y_test, predicted)

In [ ]:
a[1]